# Diseño de acople sintonizado

In [1]:
import sympy as sp
from sympy.core.compatibility import iterable
import numpy as np

# Funciones auxiliares
def mem2mem(equation,opp):
    """mem2mem:
        Aplica una operacion sencilla en ambos miembros de una ecuacion.

    Args:
        equation: Ecuacion a operar.
        opp: Termino para operar en ambos miembros.

    Returns: 
        Ecuacion de resultado luego de operar.
    """
    sel=opp[0]
    expr=sp.sympify(opp[1::])
    
    if sel=="+":
        return sp.Eq(equation.lhs+expr,equation.rhs+expr)
    elif sel=="-":
        return sp.Eq(equation.lhs-expr,equation.rhs-expr)
    elif sel=="*":
        return sp.Eq(equation.lhs*expr,equation.rhs*expr)
    elif sel=="/":
        return sp.Eq(equation.lhs/expr,equation.rhs/expr)

def disp(ecuaciones,titulo=""):
    """disp:
        Mostrar una ecuacion con titulo y separadores

    Args:
        ecuaciones: Ecuacion para imprimir
        titulo: Titulo opcional a mostrar. Defaults to "".
    """
    print(titulo + "\n")
    
    if type(ecuaciones)!=iterable:
        sp.pprint(ecuaciones,num_columns=200)
    
    else:
        for ecc in ecuaciones:
            sp.pprint(ecc,num_columns=200)

    print("\n" + "-"*80)

## Parametros

In [130]:
# frec de resonancia, 1(ultimoNumDNI) MHz
fo=16E6
# ancho de banda, 10%fo
BW=10*fo/100
# impedancias
Ra=50
RL=1E3

# asumir una CT inicial para cumplir la fo, tratando de crear un inductor cuadrado
# CT=200E-12
CT=188E-12

print(f"requerimientos: \n\tfrec de resonancia = {fo/1E6} MHz, BW = {BW/1E6} MHz \n\timpedancias entrada/salida {Ra} Ohm/{RL/1E3} kOhm\n")
print(f"se adopta un CT de {CT/1E-12} pF")

requerimientos: 
	frec de resonancia = 16.0 MHz, BW = 1.6 MHz 
	impedancias entrada/salida 50 Ohm/1.0 kOhm

se adopta un CT de 188.0 pF


## Sacar L (teorico)

In [136]:
# despejando LT
""" 1/(4 * pi^2 * CT * fo^2) """

LT=1/(4*np.pi**2 * CT * fo**2)
print(f"\nL(teorico) = {LT/1E-6} uH.")

# pasar inductancia a [uH]
LT=LT/1E-6


L(teorico) = 0.526311002131492 uH.


## Diseño del inductor (curvas J/K)

In [140]:
# diametro del cobre [mm]
Dcu=1.2E-3
# diametro del nucleo de aire [cm]
Dnuc=1

# diametro total del inductor [cm]
Dind=Dnuc+Dcu/1E-2

print(f"para inductor de {LT} uH, con nucleo de {Dnuc} cm y cobre de {Dcu/1E-3} mm")
print(f"\ttenemos un diametro de inductor de {Dind} cm.\n")

# criterio de separacion optima de Butterworth [vue/cm]
""" Nsop = 1/(1.41 * Dcu[cm])"""
Nsop=1/(1.41 * (Dcu/1E-2))
# para armarlo se separa con el mismo alambre y se apreta hasta llegar al largo calculado

# ----------------------------------------------------------------------------------
# diseño, ecuacion K de nagaoka
print("-"*60)
print("Diseño, ecuacion K de Nagaoka:")
print("-"*60)

""" L[uH] = (Dind[cm])^3 * (Nsop[vue/cm])^2 * K * 1E-3 """

""" K == 1000.0 * LT/((Dind[cm])^3 * (Nsop[vue/cm]^2)) """
K=1000.0*LT/((Dind**3) * (Nsop**2))
print(f"Coeficiente K: {K} ({np.log10(K)} en log).\n")

# entrar a grafica K
relLD=np.float32(input("relacion l/D de la tabla >>"))
print(f"Entrando al g:J/K para el K de Nagaoka, nos da la relacion l/Dind: {relLD} aprox.\n")

# largo del inductor teorico
l=relLD*Dind
# numero de espiras teorico
N=Nsop*l

print(f"tenemos un inductor de {l} cm de largo y {N} vueltas.\n".capitalize())

# correjir numero de espiras
N=np.float32(input("redondear numero de vueltas >> "))
relLD=l/Dind

print(f"corrijiendo a {N} espiras, tenemos la nueva relacion l/Dind {relLD}.\n".capitalize())

# ----------------------------------------------------------------------------------
# verificacion, ecuacion J de Nagaoka
print(f"{'-'*60} \nVerificacion, ecuacion J de Nagaoka: \n{'-'*60}")

J=np.float32(input(f"J para relacion l/D = {relLD} (log {np.log10(relLD)})"))

""" L[uH] = N^2 * (Dind [cm])/2 * J * 1E-3 """
LT=(N**2) * Dind/2 * J * 1E-3
print(f"Entrando al g:J/K para la relacion l/Dind {relLD}, da un J de {J}.\nQuedando un inductor de {LT} uH.\n")

# inductancia teorica
""" L=(uo * Aind * N^2)/l """
Aind=np.pi*((Dind*1E-2)/2)**2
print(f"inductancia teorica de: {( ((4*np.pi*1E-7) * Aind * N**2)/(l*1E-2) )/1E-6} uH")

# inductancia a [H]
LT=LT*1E-6

para inductor de 0.5171039826965331 uH, con nucleo de 1 cm y cobre de 1.2 mm
	tenemos un diametro de inductor de 1.1199999999999999 cm.

------------------------------------------------------------
Diseño, ecuacion K de Nagaoka:
------------------------------------------------------------
Coeficiente K: 10.537183231585727 (1.0227245320843135 en log).

Entrando al g:J/K para el K de Nagaoka, nos da la relacion l/Dind: 1.350000023841858 aprox.

Tenemos un inductor de 1.5120000267028808 cm de largo y 8.936170370584403 vueltas.

Corrijiendo a 9.0 espiras, tenemos la nueva relacion l/dind 1.350000023841858.

------------------------------------------------------------ 
Verificacion, ecuacion J de Nagaoka: 
------------------------------------------------------------
Entrando al g:J/K para la relacion l/Dind 1.350000023841858, da un J de 11.399999618530273.
Quedando un inductor de 0.5171039826965331 uH.

inductancia teorica de: 0.6632374040400109 uH


## Correjir C

In [141]:
# ajustar nuevo valor de C
""" CT == 1/(4 * pi^2 * LT * fo^2) """

CT=1/(4* (np.pi**2) * LT * (fo**2))

print(f"\ncorrejido a CT = {CT/1E-12} pF.\n")

# resonancia
print(f"\tresonancia en {(1/(2*np.pi*np.sqrt(LT*CT))/1E6)}  MHz\n")


correjido a CT = 191.34733382780394 pF.

	resonancia en 16.0  MHz



## Calcular R de perdidas segun nagaoka

In [142]:
# Q decargado del inductor
""" Qd == 8850 * sqrt(fo[MHz]) * (Dind[cm] * l[cm])/(102*l[cm] + 45*Dind[cm]) """
Qd=8850 * np.sqrt(fo/1E6) * (Dind*l)/(102*l + 45*Dind)

# resistencia de perdida
""" Rp = Qd * 2*pi*fo * L """
Rp=Qd * 2*np.pi*fo * LT

print(f"resistencia de perdidas {Rp/1E3} KOhm\n")

resistencia de perdidas 15.229801447489223 KOhm



## Sacar resistencia total

In [143]:
# factor Q para RLC en paralelo es inverso al RLC serie
""" Q == RT/XLo == fo/BW"""
Qparal=fo/BW

# resistencia total para este BW
RT=Qparal * 2*np.pi*fo * LT

print(f"para un Qparalelo = {Qparal}, tenemos una RT de {RT}")

para un Qparalelo = 10.0, tenemos una RT de 519.8496234180647


## Buscar capacitores C1-4

In [144]:
# transformando circuito y reflejando al primario
""" RT == rrRa // rrRL // Rp """
"""     rrRa == Ra * (1 + C2/C1)^2 == 2*RT """
"""     rrRL // Rp == 1/(1/rrRL + 1/Rp) == 2*RT"""
"""         rrRL == RL * (1 + C4/C3)^2 """

# capacidad total
""" CT == C1///C2 + C3///C4 == CT/2 + CT/2 """

# sistema de ecuaciones, buscar C1 y C2
""" C1///C2 == 1/(1/C1 + 1/C2) == CT/2 """
""" rrRa == Ra * (1 + C2/C1)^2 == 2*RT """

# resolviendo a mano
""" C2 == CT/2 * sqrt(2*RT/Ra) """
""" C1 == C2/(sqrt(2*RT/Ra) - 1) """

C2=CT/2 * np.sqrt(2*RT/Ra)
C1=C2/(np.sqrt(2*RT/Ra) - 1)

print(f"resolviendo, C1 = {C1/1E-12} pF y C2 = {C2/1E-12} pF")

print(f"\tverif: {1/(1/C1 + 1/C2)} == {CT/2}, {Ra * (1 + C2/C1)**2} == {2*RT}\n")

# sistema de ecuaciones, buscar C3 y C4
""" C3///C4 == 1/(1/C3 + 1/C4) == CT/2 """
""" rrRL // Rp == 1/(1/rrRL + 1/Rp) == 2*RT"""
"""     rrRL == RL * (1 + C4/C3)^2 """

# resolviendo a mano
""" rrRL == 2*RT/(1 - 2*RT/Rp) """
""" C4 == CT/2 * sqrt(rrRL/RL) """
""" C3 == C4/(sqrt(rrRL/RL) - 1) """

rrRL=2*RT/(1 - 2*RT/Rp)
C4=CT/2 * np.sqrt(rrRL/RL)
C3=C4/(np.sqrt(rrRL/RL) - 1)

print(f"resolviendo, C3 = {C3/1E-12} pF y C4 = {C4/1E-12} pF")

print(f"\tverif: {1/(1/C3 + 1/C4)} == {CT/2}, {1/(1/Rp + 1/((RL * (1 + C4/C3)**2)))} == {2*RT}\n")

print(f"{RL * (1 + C4/C3)**2}")


resolviendo, C1 = 122.54797382657193 pF y C2 = 436.27595930034664 pF
	verif: 9.567366691390197e-11 == 9.567366691390197e-11, 1039.6992468361293 == 1039.6992468361293

resolviendo, C3 = 1793.4920956065616 pF y C4 = 101.06496811906118 pF
	verif: 9.567366691390197e-11 == 9.567366691390197e-11, 1039.6992468361295 == 1039.6992468361293

1115.877311559431


## Normalizar componentes

In [145]:
# si se normalizan las capacidades
C1n=120E-12
C2n=390E-12 + 47E-12
C3n=1800E-12
C4n=100E-12

# capacidad total
CTn=(1/(1/C1n + 1/C2n)) + (1/(1/C3n + 1/C4n))
print(f"capacidad normalizado total {CTn/1E-12} pF")

# resonando a 
fon=1/(2*np.pi * np.sqrt(LT*CTn))
print(f"\tresonando a {fon/1E6} MHz")

# factor Qparal
rrRa=Ra*(1 + C2n/C1n)**2
rrRL=RL*(1 + C4n/C3n)**2
RT=1/(1/Rp + 1/rrRL + 1/rrRa)

Qparal=RT/(2*np.pi*fon * LT)
print(f"\tfactor de merito {Qparal}")

capacidad normalizado total 188.8840593404517 pF
	resonando a 16.10399164131311 MHz
	factor de merito 10.104454217274471


### ECUACIONES RELOCAS AUXILIARES

In [60]:
# frecuencia de resonancia, sacar LT
""" fo = 1/(2*pi * sqrt(LT*CT)) """
ssfo,ssLT,ssCT=sp.symbols("ssfo,ssLT,ssCT")

ec0=sp.Eq(ssfo,1/(2*sp.pi*sp.sqrt(ssLT*ssCT)))
sp.pprint(ec0)

print(f"\nresolviendo, LT == {sp.solve(ec0,ssLT)}\n")
print("-"*60)
# ----------------------------------------------------------------------------

# ecuacion de nagaoka, sacar K
""" L[uH] = (Dind[cm])^3 * (Nsop[vue/cm])^2 * K * 1E-3 """
ssLT,ssDind,ssNsop,ssK=sp.symbols("ssLT,ssDind,ssNsop,ssK")

ec1=sp.Eq(ssLT,ssDind**3 * ssNsop**2 * ssK * 1E-3)
sp.pprint(ec1)

print(f"\nresolviendo, K == {sp.solve(ec1,ssK)}\n")
print("-"*60)
# ----------------------------------------------------------------------------

# frecuencia de resonancia, sacar CT
""" fo = 1/(2*pi * sqrt(LT*CT)) """
sp.pprint(ec0)

print(f"\nresolviendo, CT == {sp.solve(ec0,ssCT)}\n")
print("-"*60)
# ----------------------------------------------------------------------------


""" RT == rrRa // rrRL // Rp """
"""     rrRa == Ra * (1 + C2/C1)^2 == 2*RT """
"""     rrRL // Rp == 1/(1/rrRL + 1/Rp) == 2*RT"""
"""         rrRL == RL * (1 + C4/C3)^2 """


               1        
ssfo = ─────────────────
             ___________
       2⋅π⋅╲╱ ssCT⋅ssLT 

resolviendo, LT == [1/(4*pi**2*ssCT*ssfo**2)]

------------------------------------------------------------
                   3           2
ssLT = 0.001⋅ssDind ⋅ssK⋅ssNsop 

resolviendo, K == [1000.0*ssLT/(ssDind**3*ssNsop**2)]

------------------------------------------------------------
               1        
ssfo = ─────────────────
             ___________
       2⋅π⋅╲╱ ssCT⋅ssLT 

resolviendo, CT == [1/(4*pi**2*ssLT*ssfo**2)]

------------------------------------------------------------


'         rrRL == RL * (1 + C4/C3)^2 '